In [38]:
!pip install xgboost

In [39]:
import numpy as np
import pandas as pd
from datetime import datetime

In [40]:
# Data ingestion:
prec  = pd.read_csv("Daily Precipitation.csv", encoding = "utf-8")
moist = pd.read_csv("Daily Soil Mositure.csv", encoding = "utf-8")
temp  = pd.read_csv("Daily Temperature.csv", encoding = "utf-8")
ndvi  = pd.read_csv("Eight Day NDVI.csv", encoding = "utf-8")
prod  = pd.read_csv("Production Quantity.csv", encoding = "utf-8")

# Panel data. Some data is missing, and we will use interpolation to fill the blanks. 

# Data notes:
# prec  : daily   | long | no imputing
# moist : daily   | long | interp imputing
# temp  : daily   | long | interp imputing
# ndvi  : weekly  | long |
# prod  : monthly | long | 2015-1 ~ 2020-12

# The training set only has data frequency of month, so we have to engineer features of monthly frequency. 

In [41]:
def getDatetime(s): # datetime string parser, specific to data provided. 
    return datetime.strptime(s, "%Y-%m-%dT%H:%M:%S.000Z")

def neatDate(df):
    df.loc[:,"date"] = df.start_date.apply(getDatetime)
    df.drop([col for col in df.columns if "_date" in col], axis = 1, inplace = True)
    # Create useful time columns
    for col in ["year", "month"]:
        df.loc[:,col] = df.date.apply(lambda x : eval(f"x.{col}"))
    return df

In [42]:
# Preprocessing and data cleaning:

# Cleaning daily freq data
ndvi  = pd.read_csv("Eight Day NDVI.csv", encoding = "utf-8")
daily = pd.merge(prec, moist, how = "outer", on = ["start_date", "region_id"]) # outer merge to preserve maximum amount of data 
daily = pd.merge(daily, temp, how = "outer", on = ["start_date", "region_id"])

daily = neatDate(daily) # create time columns. See function above. 

daily.set_index("date",inplace=True) # set datetime index to use time interpolation method. 
daily.loc[:,"temp_interp"] = daily.groupby(by = "region_id").temp.apply(lambda x: x.interpolate(method = "time")).ffill().bfill() # time-series data --> using time method to interpolate and impute. 
daily.loc[:,"smos_interp"] = daily.groupby(by = "region_id").smos.apply(lambda x: x.interpolate(method = "time")).ffill().bfill() # bfill only to treat missing entries at the beginning of data. Negligible bias. 
daily.reset_index(inplace = True) # reset index to integers. 

# Cleaning ndiv data
ndvi = neatDate(ndvi)
ndvi.set_index("date", inplace = True)
ndvi.ndvi = ndvi.groupby(by = "region_id").ndvi.apply(lambda x : x.interpolate(method = "time").ffill().bfill())
ndvi.reset_index(inplace = True)

# Cleaning target data
prod = neatDate(prod)


In [43]:
# Feature engineering
# All features monthly. 

# Generating features for daily data
data = pd.DataFrame()

def gbo(df):
    return df.groupby(by = ["region_id", "year", "month"])

gbo_daily = gbo(daily)

features_dict = {
    "precip" : ["sum", 
                "max", 
               ], 
    "temp" : ["mean", 
              "max", 
             ] ,
    "smos" : ["mean", 
              "max", 
             ]
}

for k in features_dict:
    for v in features_dict[k]:
        data.loc[:,f"{k}_{v}"] = eval(f"gbo_daily.{k}.{v}()").ffill().bfill()
        
# Generating lag features. In this way, we are not restricted to time series forecasting, but can use folds. 

lag_vars = ["precip_sum", "temp_mean", "smos_mean", 
               "precip_max", "temp_max", "smos_max", 
]
max_lag = 2
for col in lag_vars:
    data.loc[:,f"{col}_pctchg"] = data.loc[:,col].groupby(by = "region_id").pct_change().bfill()
    for i in range(1,max_lag):
        data.loc[:,f"{col}_lag_{i+1}"] = data.loc[:,col].groupby(by = "region_id").shift(i+1).bfill()

data.reset_index(inplace = True)

# Generating features for ndvi data

gbo_ndvi = gbo(ndvi)
data_ndvi = pd.DataFrame()


data_ndvi.loc[:,"ndvi_mean"] = gbo_ndvi.ndvi.mean()
data_ndvi.loc[:,"ndvi_mean_pctchg" ] = data_ndvi.ndvi_mean.groupby(by = "region_id").pct_change().bfill()
data_ndvi.reset_index(inplace = True)

data = pd.merge(left = data, right = data_ndvi, how = "inner", on = ["region_id","year","month"])
train = pd.merge(left = data, right = prod, how = "inner", on = ["region_id","year","month"])

# Categori-fy
for col in ["month", "region_id"]:
    train.loc[:,col] = train.loc[:,col].astype("category")

# IMPORTANT ASSUMPTION: Gople syrup is a agricultural product, and thus its production is follows seasonality
# Some seasonal effects may not be captured by the continuous variables provided, and thus we include month 
# as a categorical variable to account for these effects. 

train
# Data ingestion, imputation, cleaning, engineering complete. Proceede to training.

,region_id,year,month,precip_sum,precip_max,temp_mean,temp_max,smos_mean,smos_max,precip_sum_pctchg,...,precip_max_pctchg,precip_max_lag_2,temp_max_pctchg,temp_max_lag_2,smos_max_pctchg,smos_max_lag_2,ndvi_mean,ndvi_mean_pctchg,prod,date
0,93,2015,1,136.228017,54.497266,25.003593,27.547240,0.318552,0.515493,-0.641584,...,0.051539,18.894332,0.026980,30.043779,0.267678,0.407483,0.761228,0.015708,171725,2015-01-01
1,93,2015,2,33.771850,15.357199,26.807568,30.535201,0.291452,0.401876,-0.752093,...,-0.718202,51.826174,0.108467,26.823536,-0.220404,0.406643,0.758957,-0.002983,188325,2015-02-01
2,93,2015,3,107.094222,13.704099,26.954109,30.910728,0.251969,0.331012,2.171109,...,-0.107643,54.497266,0.012298,27.547240,-0.176334,0.515493,0.752754,-0.008173,247856,2015-03-01
3,93,2015,4,313.994152,35.989889,26.948865,29.560593,0.248915,0.319016,1.931943,...,1.626213,15.357199,-0.043679,30.535201,-0.036238,0.401876,0.785057,0.042912,282791,2015-04-01
4,93,2015,5,279.247567,29.155820,25.405121,29.758767,0.280602,0.401614,-0.110660,...,-0.189889,13.704099,0.006704,30.910728,0.258914,0.331012,0.805262,0.025737,291057,2015-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,105,2020,8,236.695865,26.458364,27.424585,29.364843,0.225784,0.393348,-0.218464,...,-0.431745,25.339259,0.016449,28.165768,0.132425,0.726186,0.813444,-0.010391,57818,2020-08-01
712,105,2020,9,205.578907,16.659523,27.940492,30.178225,0.277477,0.467299,-0.131464,...,-0.370349,46.560750,0.027699,28.889628,0.188004,0.347350,0.810768,-0.003291,57474,2020-09-01
713,105,2020,10,349.576941,47.817073,27.048770,29.820700,0.277670,0.619990,0.700451,...,1.870255,26.458364,-0.011847,29.364843,0.326753,0.393348,0.815188,0.005452,51821,2020-10-01
714,105,2020,11,530.295923,77.648337,24.715070,26.869159,0.302014,0.606885,0.516965,...,0.623862,16.659523,-0.098976,30.178225,-0.021138,0.467299,0.803192,-0.014715,44947,2020-11-01


In [44]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, LinearRegression

from sklearn.model_selection import cross_validate, TimeSeriesSplit, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_percentage_error, r2_score

import xgboost as xgb 
from xgboost import XGBRegressor, XGBRFRegressor


X = train.loc[:,[col for col in train.columns if "_" in col or col == "month"]]
y = train.loc[:,"prod"]


def evaluate_model(model, X, y, cv, metric):
    scorer = make_scorer(metric, greater_is_better=False, squared=False)
    return np.mean(cross_validate(model, X, y, cv=cv, scoring=scorer, n_jobs=-1)['test_score'])

pipeline = Pipeline([
    ('scale', StandardScaler()),
    ("model", XGBRegressor(n_estimators = 100, max_depth = 5, objective = "reg:squarederror"))
])

# Notes: 
# min: -37494.759403383185
# param: kfold = 20, not tuned, xgbr
# tuned: -32503.77803497381
# param: {'model__learning_rate': 0.06, 'model__max_depth': 5, 'model__reg_alpha': 0.02, 'model__reg_lambda': 0.01}

metric = mean_squared_error
scorer = make_scorer(metric, greater_is_better=False, squared=False)
cv = KFold(n_splits=20, shuffle=True)
search = GridSearchCV(pipeline, {
    'model__learning_rate': [0.05, 0.06, 0.07], 
    'model__reg_alpha': [0.01, 0.015, 0.02], 
    'model__reg_lambda': [0.005, 0.01, 0.015]
}, scoring=scorer, refit=True, cv=cv, n_jobs=-1)
search.fit(X, y)

# To reader: the GridSearchCV grid is not random... Multiple experiments were 
# made, and only the last set were recorded here for your reference. 


GridSearchCV(cv=KFold(n_splits=20, random_state=None, shuffle=True),
             estimator=Pipeline(steps=[('scale', StandardScaler()),
                                       ('model',
                                        XGBRegressor(max_depth=5,
                                                     objective='reg:squarederror'))]),
             n_jobs=-1,
             param_grid={'model__learning_rate': [0.05, 0.06, 0.07],
                         'model__reg_alpha': [0.01, 0.015, 0.02],
                         'model__reg_lambda': [0.005, 0.01, 0.015]},
             scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False))

In [45]:
print(search.best_params_)
best_model = search.best_estimator_
evaluate_model(best_model, X, y,cv = cv, metric = mean_squared_error)

{'model__learning_rate': 0.06, 'model__reg_alpha': 0.02, 'model__reg_lambda': 0.005}


-33627.05857694438

In [46]:
# Make prediction. 

pred = pd.read_csv("predicted_production_qty.csv", encoding = "utf-8")
pred_b = neatDate(pred)
test = pd.merge(left = data, right = pred_b, how = "inner", on = ["region_id","year","month"])
X_test = test.loc[:,[col for col in test.columns if "_" in col or col == "month"]]
prediction = best_model.predict(X_test)
pred = pd.read_csv("predicted_production_qty.csv", encoding = "utf-8")
pred.loc[:,"prod"] = prediction
pred.to_csv("linruoyu1202@outlook.com.csv", index = False, encoding = "utf-8")